In [2]:
import numpy as np
import random

def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

def cost_function(network, test_data, onehot=True):
    c = 0
    for example, y in test_data:
        if not onehot:
            y = np.eye(3,1, k=-int(y))
        yhat = network.feedforward(example)
        c += np.sum((y - yhat)**2)
    return c / len(test_data)

In [4]:
class Network:

    def __init__(self, sizes, output=True):
        """
        Список ``sizes`` содержит количество нейронов в соответствующих слоях
        нейронной сети. К примеру, если бы этот лист выглядел как [2, 3, 1],
        то мы бы получили трёхслойную нейросеть, с двумя нейронами в первом
        (входном), тремя нейронами во втором (промежуточном) и одним нейроном
        в третьем (выходном, внешнем) слое. Смещения и веса для нейронных сетей
        инициализируются случайными значениями, подчиняющимися стандартному нормальному
        распределению. Обратите внимание, что первый слой подразумевается слоем, 
        принимающим входные данные, поэтому мы не будем добавлять к нему смещение 
        (делать это не принято, поскольку смещения используются только при 
        вычислении выходных значений нейронов последующих слоёв)
        """

        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]
        self.output = output

    def feedforward(self, a):
        """
        Вычислить и вернуть выходную активацию нейронной сети
        при получении ``a`` на входе (бывшее forward_pass).
        """
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """
        Обучить нейронную сеть, используя алгоритм стохастического
        (mini-batch) градиентного спуска. 
        ``training_data`` - лист кортежей вида ``(x, y)``, где 
        x - вход обучающего примера, y - желаемый выход (в формате one-hot). 
        Роль остальных обязательных параметров должна быть понятна из их названия.
        Если предоставлен опциональный аргумент ``test_data``, 
        то после каждой эпохи обучения сеть будет протестирована на этих данных 
        и промежуточный результат обучения будет выведен в консоль. 
        ``test_data`` -- это список кортежей из входных данных 
        и номеров правильных классов примеров (т.е. argmax(y),
        если y -- набор ответов в той же форме, что и в тренировочных данных).
        Тестирование полезно для мониторинга процесса обучения,
        но может существенно замедлить работу программы.
        """

        if test_data is not None: n_test = len(test_data)
        n = len(training_data)
        success_tests = 0
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data is not None and self.output:
                success_tests = self.evaluate(test_data)
                print("Эпоха {0}: {1} / {2}".format(
                    j, success_tests, n_test))
            elif self.output:
                print("Эпоха {0} завершена".format(j))
        if test_data is not None:
            return success_tests / n_test

    def update_mini_batch(self, mini_batch, eta):
        """
        Обновить веса и смещения нейронной сети, сделав шаг градиентного
        спуска на основе алгоритма обратного распространения ошибки, примененного
        к одному mini batch.
        ``mini_batch`` - список кортежей вида ``(x, y)``,
        ``eta`` - величина шага (learning rate).
        """
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            
        eps = eta / len(mini_batch)
        self.weights = [w - eps * nw for w, nw in zip(self.weights, nabla_w)]
        self.biases  = [b - eps * nb for b, nb in zip(self.biases,  nabla_b)]
        
    def backprop(self, x, y):
        """
        Возвращает кортеж ``(nabla_b, nabla_w)`` -- градиент целевой функции по всем параметрам сети.
        ``nabla_b`` и ``nabla_w`` -- послойные списки массивов ndarray,
        такие же, как self.biases и self.weights соответственно.
        """
        # Эту функцию необходимо реализовать
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        # прямое распространение (forward pass)
        a = [np.zeros((s,1)) for s in self.sizes]
        a[0] = np.array(x)


        for i, b, w in zip(range(1,len(a)),self.biases, self.weights):
            a[i] = sigmoid(np.dot(w,a[i-1]) + b)

        delta =  (a[-1] - np.array(y)) * a[-1]*(1-a[-1])# ошибка выходного слоя
        nabla_b[-1] =  delta # производная J по смещениям выходного слоя
        nabla_w[-1] =  delta.dot(a[-2].T) # производная J по весам выходного слоя

        for l in range(2, self.num_layers):    
            delta = self.weights[-l+1].T.dot(delta) * a[-l]*(1-a[-l])
            nabla_b[-l] =  delta# производная J по смещениям L-l-го слоя
            nabla_w[-l] =  delta.dot(a[-l-1].T)# производная J по весам L-l-го слоя

        return nabla_b, nabla_w

    def evaluate(self, test_data):
        """
        Вернуть количество тестовых примеров, для которых нейронная сеть
        возвращает правильный ответ. Обратите внимание: подразумевается,
        что выход нейронной сети - это индекс, указывающий, какой из нейронов
        последнего слоя имеет наибольшую активацию.
        """
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """
        Возвращает вектор частных производных (\partial C_x) / (\partial a) 
        целевой функции по активациям выходного слоя.
        """
        return (output_activations-y)
    
data = np.loadtxt("data.csv", delimiter=",")

means = data.mean(axis=0)
means[-1] = 0  # правильные ответы мы нормализовывать не будем: это качественные переменные
stds = data.std(axis=0)
stds[-1] = 1
data = (data - means) / stds

np.random.seed(42)
test_index = np.random.choice([True, False], len(data), replace=True, p=[0.25, 0.75])
test  = data[test_index]
train = data[np.logical_not(test_index)]

# eye - чтобы создать вертикальный вектор, аналогичный тому, который будет выдавать нейросеть на выходе
train = [(d[:3][:, np.newaxis], np.eye(3, 1, k=-int(d[-1]))) for d in train]  
test =  [(d[:3][:, np.newaxis], d[-1]) for d in test]

input_count  = 3  # 3 нейрона входного слоя
hidden_count = 6  # 6 нейронов внутреннего слоя
output_count = 3  # 3 нейрона выходного слоя, по индикатору для каждого из классов "недолёт", "попал" и "перелёт"

random.seed(1)
np.random.seed(1)
nn = Network([input_count, hidden_count, output_count])
nn.SGD(training_data=train, epochs=100, mini_batch_size=5, eta=1, test_data=test)


Эпоха 0: 74 / 130
Эпоха 1: 83 / 130
Эпоха 2: 91 / 130
Эпоха 3: 92 / 130
Эпоха 4: 95 / 130
Эпоха 5: 90 / 130
Эпоха 6: 93 / 130
Эпоха 7: 95 / 130
Эпоха 8: 95 / 130
Эпоха 9: 97 / 130
Эпоха 10: 98 / 130
Эпоха 11: 97 / 130
Эпоха 12: 100 / 130
Эпоха 13: 100 / 130
Эпоха 14: 103 / 130
Эпоха 15: 103 / 130
Эпоха 16: 103 / 130
Эпоха 17: 103 / 130
Эпоха 18: 103 / 130
Эпоха 19: 103 / 130
Эпоха 20: 103 / 130
Эпоха 21: 103 / 130
Эпоха 22: 103 / 130
Эпоха 23: 101 / 130
Эпоха 24: 103 / 130
Эпоха 25: 103 / 130
Эпоха 26: 103 / 130
Эпоха 27: 103 / 130
Эпоха 28: 104 / 130
Эпоха 29: 103 / 130
Эпоха 30: 104 / 130
Эпоха 31: 104 / 130
Эпоха 32: 104 / 130
Эпоха 33: 103 / 130
Эпоха 34: 103 / 130
Эпоха 35: 105 / 130
Эпоха 36: 104 / 130
Эпоха 37: 103 / 130
Эпоха 38: 104 / 130
Эпоха 39: 104 / 130
Эпоха 40: 103 / 130
Эпоха 41: 105 / 130
Эпоха 42: 103 / 130
Эпоха 43: 104 / 130
Эпоха 44: 105 / 130
Эпоха 45: 106 / 130
Эпоха 46: 103 / 130
Эпоха 47: 106 / 130
Эпоха 48: 112 / 130
Эпоха 49: 103 / 130
Эпоха 50: 105 / 130
Эп

0.8615384615384616